In [11]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from bjtu_programming.search_rs import DATA_DIR
from bjtu_programming.search_rs.feature_engineering.time import DateUtil, DateConverter
from bjtu_programming.search_rs.models.xgb_lightgbm import XgbLightGBWrapper
from bjtu_programming.search_rs.utils.pd_util import PandasUtil

/Users/chubin.zheng/Documents/PyCharmProjects/learning/programing/bjtu_programming/search_rs/data


In [12]:
# train: 81680000+
_file = DATA_DIR.joinpath("train_info_100000.txt")
# _file = DATA_DIR.joinpath("train_info_5000000.txt")
# _file = DATA_DIR.joinpath("train_info.txt")
train_df = PandasUtil.read_csv_by_dd(_file, separator="\t")
print(train_df.npartitions)
train_df = train_df.repartition(npartitions=train_df.npartitions * 5)
print(train_df.npartitions)

train_df.columns =[
    "user_id", "item_id", "ctx_time_expose", "ctx_net_env", "u_cnt_flush",
    "ctx_show_pos", "is_click", "cnt_read_duration"
]

# https://docs.dask.org/en/stable/generated/dask.dataframe.Series.apply.html?highlight=apply
train_df['ctx_time_expose'] = train_df['ctx_time_expose'].apply(DateConverter.timestamp_to_datetime, meta=('ctx_time_expose', 'datetime64[ns]'))
train_df['ctx_time_expose_day_of_week'] = train_df['ctx_time_expose'].apply(DateUtil.get_day_of_week, meta=('ctx_time_expose', 'int8'))
train_df['ctx_time_expose_hour_bucket'] = train_df['ctx_time_expose'].apply(DateUtil.get_hour_bucket, meta=('ctx_time_expose', 'int8'))



# train_df['ctx_time_expose'] = train_df['ctx_time_expose'].apply(lambda x: DateConverter.timestamp_to_datetime(x))
# train_df['ctx_time_expose_day_of_week'] = train_df['ctx_time_expose'].apply(lambda x: DateUtil.get_day_of_week(x))
# train_df['ctx_time_expose_hour_bucket'] = train_df['ctx_time_expose'].apply(lambda x: DateUtil.get_hour_bucket(x))

# train_df = train_df.sort_values(by=['ctx_time_expose'], ascending=True)

print(train_df.shape)
train_df.head()

1
5
(Delayed('int-f6ebebd5-3a18-464e-b783-348c191ba78d'), 10)


,user_id,item_id,ctx_time_expose,ctx_net_env,u_cnt_flush,ctx_show_pos,is_click,cnt_read_duration,ctx_time_expose_day_of_week,ctx_time_expose_hour_bucket
0,1000014754,463852707,2021-06-28 09:29:16.147000+08:00,5,0,13,1,80,0,2
1,1000014754,463067100,2021-06-27 09:25:47.178000+08:00,5,0,13,0,0,6,2
2,1000014754,463625484,2021-06-27 10:54:06.340000+08:00,5,0,12,1,268,6,2
3,1000014754,463625484,2021-06-27 10:54:06.340000+08:00,5,0,12,1,57,6,2
4,1000014754,463420121,2021-06-26 15:21:07.329000+08:00,2,0,14,0,0,5,2


In [3]:
item_df = pd.read_csv(DATA_DIR.joinpath("news_info.txt"),
                      sep='\t',
                      header=None)
item_df.columns =[
    "item_id", "i_title", "i_time_release", "i_cnt_img", "i_category1", "i_category2", "i_keys_prob"
]

label_encoders = {
    "i_category1": LabelEncoder(),
    "ctx_device": LabelEncoder(),
    "ctx_os": LabelEncoder(),
    "ctx_province": LabelEncoder(),
    "ctx_city": LabelEncoder(),
}
item_df["i_category1"] = label_encoders.get("i_category1").fit_transform(item_df["i_category1"])

print(item_df.shape)
item_df.head(1000)

(347466, 7)


,item_id,i_title,i_time_release,i_cnt_img,i_category1,i_category2,i_keys_prob
0,325279629,合集|外交部每日答问,1.624890e+12,0.0,38,NaN,NaN
1,334493096,汽车点评网友“毛蒙”分享的小视频“,1.563405e+12,0.0,22,汽车/其他,https:8.135272
2,341014580,"抖音爆红车模小姐姐,可爱如神仙,如今大火的她还玩起这个!",1.567068e+12,3.0,10,娱乐/其他,"初恋:11.052198,吊坠:9.525825,吊坠^^耳环:7.982950,妆容:10..."
3,344297472,"一夜能挣30万的网红车模,她的捞金神器竟是这个!",1.569240e+12,3.0,10,娱乐/其他,"主播:10.299306,发展:7.355088,可爱:8.705284,清纯:10.108..."
4,347286095,"男子购买假号牌被查,后备箱找到真车牌,车主直言不愿太高调",1.571271e+12,3.0,22,汽车/用车,"买车:9.357286,交警:8.964902,交通:7.790495,交通^^秩序:10...."
...,...,...,...,...,...,...,...
995,461750018,越野车内竟然藏了个600升大油罐苏州交通执法查获“流动炸弹”,1.624321e+12,3.0,20,案件/法治案件,"中石化:10.730155,公安:8.260727,加油车:16.895083,协查通报:1..."
996,461750027,"也许他才是神雕中剑法最高的人,实力超杨过却非独孤求败!",1.624321e+12,4.0,16,文化艺术/读书,"一灯大师:16.885216,九阴真经:11.781893,人才:7.700833,佩剑:8..."
997,461750057,"“雷车”也降价,品牌价值不足,那就只能靠价格弥补",1.624321e+12,9.0,22,汽车/汽车资讯,"bba:10.433508,ls:15.310630,东方人:8.539490,中网:6.1..."
998,461750099,"1分2失误3大数据垫底!男篮第一天才进nba难了,投三不沾暴露软肋",1.624321e+12,6.0,1,体育/中国篮球,"cba:8.526146,nba^^球探:12.847561,三分球:10.587807,三..."


In [4]:
user_df = pd.read_csv(DATA_DIR.joinpath("user_info.txt"),
                      sep='\t',
                      header=None)
user_df.columns =[
    "user_id", "u_device", "u_os", "u_province", "u_city", "u_age_prob", "u_sex_prob"
]

for column in ["u_device", "u_os", "u_province", "u_city"]:
    user_df[column] = label_encoders.get(column).fit_transform(user_df[column])

print(user_df.shape)
user_df.head(1000)

(1029718, 7)


,user_id,ctx_device,ctx_os,ctx_province,ctx_city,u_age_prob,u_sex_prob
0,17340,2641,1,184,290,"A_0_24:0.029774,A_25_29:0.126834,A_30_39:0.789...","female:0.000000,male:1.000000"
1,394666,160,0,206,570,"A_0_24:0.400583,A_25_29:0.261192,A_30_39:0.268...","female:0.000000,male:1.000000"
2,413322,2643,1,199,339,"A_0_24:0.000076,A_25_29:0.017034,A_30_39:0.972...","female:0.191682,male:0.808318"
3,450280,1380,0,243,501,"A_0_24:0.943772,A_25_29:0.043366,A_30_39:0.011...","female:0.198083,male:0.801917"
4,456646,1167,0,223,297,"A_0_24:0.024724,A_25_29:0.230098,A_30_39:0.409...","female:0.335419,male:0.664581"
...,...,...,...,...,...,...,...
995,23199678,1152,0,219,590,"A_0_24:0.049074,A_25_29:0.629261,A_30_39:0.302...","female:0.000000,male:1.000000"
996,23224752,241,0,206,363,"A_0_24:0.008884,A_25_29:0.682589,A_30_39:0.307...","female:0.000000,male:1.000000"
997,23237820,1342,0,199,339,"A_0_24:0.000010,A_25_29:0.000324,A_30_39:0.055...","female:0.000000,male:1.000000"
998,23262374,1272,0,280,699,"A_0_24:0.385883,A_25_29:0.315351,A_30_39:0.242...","female:0.000000,male:1.000000"


In [5]:
X_feats = [
    ["user_id", "item_id", "ctx_time_expose_day_of_week", "ctx_time_expose_hour_bucket",  "ctx_net_env", "u_cnt_flush"],
    ["item_id", "i_category1", "i_cnt_img"],
    ["user_id", "ctx_device", "ctx_os", "ctx_province", "ctx_city"]
]

Y_train = train_df["is_click"]
train_df = train_df[X_feats[0]]
item_df = item_df[X_feats[1]]
user_df = user_df[X_feats[2]]

train_df = pd.merge(train_df, item_df, how='left', on='item_id')
train_df = pd.merge(train_df, user_df, how='left', on='user_id')

categories = [
    "ctx_time_expose_day_of_week", "ctx_time_expose_hour_bucket", "ctx_net_env",
    "i_category1",
    "ctx_device", "ctx_os", "ctx_province", "ctx_city"
]


In [6]:
train_size = int(train_df.shape[0] * 0.7)
X_tra, Y_tra = train_df[:train_size], Y_train[:train_size]
X_dev, Y_dev = train_df[train_size:], Y_train[train_size:]

X_tra

,user_id,item_id,ctx_time_expose_day_of_week,ctx_time_expose_hour_bucket,ctx_net_env,u_cnt_flush,i_category1,i_cnt_img,ctx_device,ctx_os,ctx_province,ctx_city
0,2207836358,462770636,3,3,2,3,28,0.0,1277,0,219,532
1,1514664904,462456703,3,3,2,1,2,3.0,1585,0,237,584
2,2224954538,462723631,3,3,2,3,30,20.0,325,0,199,339
3,1637502008,462621612,3,3,2,1,3,5.0,1574,0,237,437
4,1599561538,462753067,3,3,2,0,1,13.0,864,0,242,530
...,...,...,...,...,...,...,...,...,...,...,...,...
3499995,1485781606,463443162,0,2,2,7,12,0.0,1282,0,220,395
3499996,1485781606,463808863,0,2,2,7,16,3.0,1282,0,220,395
3499997,1485781606,463286849,0,2,2,7,11,3.0,1282,0,220,395
3499998,1481514482,463424514,0,2,5,13,1,9.0,872,0,215,653


In [7]:

XgbLightGBWrapper.create_classifier(X_tra, Y_tra, X_dev, Y_dev, cate_features=categories)

/Users/chubin.zheng/ENVS/web_search_and_recommender/lib/python3.7/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chubin.zheng/ENVS/web_search_and_recommender/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chubin.zheng/ENVS/web_search_and_recommender/lib/python3.7/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/Users/chubin.zheng/ENVS

[1]	eval's binary_logloss: 0.401294	eval's auc: 0.655256
[2]	eval's binary_logloss: 0.399643	eval's auc: 0.658362
[3]	eval's binary_logloss: 0.39818	eval's auc: 0.658409
[4]	eval's binary_logloss: 0.396863	eval's auc: 0.658901
[5]	eval's binary_logloss: 0.395654	eval's auc: 0.658954
[6]	eval's binary_logloss: 0.394563	eval's auc: 0.658976
[7]	eval's binary_logloss: 0.393541	eval's auc: 0.659632
[8]	eval's binary_logloss: 0.39264	eval's auc: 0.659824
[9]	eval's binary_logloss: 0.391792	eval's auc: 0.660354
[10]	eval's binary_logloss: 0.39101	eval's auc: 0.660805
[11]	eval's binary_logloss: 0.390303	eval's auc: 0.661129
[12]	eval's binary_logloss: 0.389652	eval's auc: 0.661414
[13]	eval's binary_logloss: 0.389066	eval's auc: 0.661572
[14]	eval's binary_logloss: 0.388512	eval's auc: 0.661807
[15]	eval's binary_logloss: 0.387996	eval's auc: 0.661963
[16]	eval's binary_logloss: 0.387507	eval's auc: 0.662299
[17]	eval's binary_logloss: 0.38707	eval's auc: 0.66246
[18]	eval's binary_logloss: 

LGBMClassifier(colsample_bytree=1, device='cpu', learning_rate=0.05,
               max_bin=425, min_child_samples=10, min_child_weight=5,
               min_split_gain=0, n_jobs=16, num_leaves=64, objective='binary',
               reg_alpha=3, reg_lambda=5, seed=1000, silent=True, subsample=0.8,
               subsample_for_bin=50000, subsample_freq=1)

In [7]:
test_file = DATA_DIR.joinpath("test_info.txt")
test_df = pd.read_csv(test_file,
                      sep='\t',
                      header=None)
test_df.columns = ["order", "user_id", "item_id", 'ctx_time_expose', "ctx_net_env", "u_cnt_flush",]

test_df['ctx_time_expose'] = test_df['ctx_time_expose'].apply(lambda x: DateConverter.timestamp_to_datetime(x))
test_df['ctx_time_expose_day_of_week'] = test_df['ctx_time_expose'].apply(lambda x: DateUtil.get_day_of_week(x))
test_df['ctx_time_expose_hour_bucket'] = test_df['ctx_time_expose'].apply(lambda x: DateUtil.get_hour_bucket(x))

test_df

,order,user_id,item_id,ctx_time_expose,ctx_net_env,u_cnt_flush,ctx_time_expose_day_of_week,ctx_time_expose_hour_bucket
0,0,2204868548,464495940,2021-06-30 08:09:34.995000+08:00,2,4,2,1
1,1,1623790870,464553715,2021-06-30 23:14:14.894000+08:00,2,2,2,3
2,2,1637010070,464687570,2021-06-30 19:45:46.711000+08:00,2,10,2,3
3,3,1634251474,464661865,2021-06-30 17:56:54.335000+08:00,2,10,2,3
4,4,2352721164,464692079,2021-06-30 14:43:03.911000+08:00,2,11,2,2
...,...,...,...,...,...,...,...,...
49995,49995,1640585246,464553715,2021-06-30 15:02:48.698000+08:00,2,1,2,2
49996,49996,1987843480,464382231,2021-06-30 07:31:09.794000+08:00,2,1,2,1
49997,49997,2391080328,464617167,2021-06-30 12:32:46.512000+08:00,5,1,2,2
49998,49998,125699342,464172118,2021-06-30 14:31:32.217000+08:00,2,11,2,2


In [9]:
test_df = test_df[X_feats[0]]

test_df = pd.merge(test_df, item_df, how='left', on='item_id')
test_df = pd.merge(test_df, user_df, how='left', on='user_id')

test_df

,user_id,item_id,ctx_time_expose_day_of_week,ctx_time_expose_hour_bucket,ctx_net_env,u_cnt_flush,i_category1,i_cnt_img,ctx_device,ctx_os,ctx_province,ctx_city
0,2204868548,464495940,2,1,2,4,10,23.0,1478,0,223,505
1,1623790870,464553715,2,3,2,2,10,9.0,2809,0,215,653
2,1637010070,464687570,2,3,2,10,10,6.0,1224,0,279,559
3,1634251474,464661865,2,3,2,10,12,9.0,980,0,241,359
4,2352721164,464692079,2,2,2,11,3,8.0,1784,0,194,621
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1640585246,464553715,2,2,2,1,10,9.0,1575,0,219,550
49996,1987843480,464382231,2,1,2,1,2,3.0,1601,0,223,509
49997,2391080328,464617167,2,2,5,1,15,6.0,863,0,241,645
49998,125699342,464172118,2,2,2,11,3,0.0,575,0,199,339


In [13]:
predicts = XgbLightGBWrapper.predict("model.txt", test_df)
predicts

y_pred.shape=(50000,), <class 'numpy.ndarray'>,[0.08377885 0.1089102  0.11215458 0.10648261 0.06874617]


array([0.08377885, 0.1089102 , 0.11215458, ..., 0.1863659 , 0.09705234,
       0.12345721])

In [17]:
df = pd.DataFrame(
    {"pred": predicts}
)
df.to_csv("predict.csv")
df

,pred
0,0.083779
1,0.108910
2,0.112155
3,0.106483
4,0.068746
...,...
49995,0.211686
49996,0.245154
49997,0.186366
49998,0.097052
